In [2]:
import networkx as nx
import matplotlib.pyplot as plt
from surface_dynamics.databases.flat_surfaces import CylinderDiagrams
from surface_dynamics import AbelianStratum
import sys
from utils import find_generic_pants, find_homologous_cylinders, valid_equivalence_classes

In [3]:
def check_sets(element, list_sets):
    for s in list_sets:
        if element in s:
            return True
    return False

def compute_singularities(perm):
    """Computes a list of singularities. 
    
    Each saddle is identified with its right vertex. A singularity is then an equivalence class (set) of saddles."""
    singularities = []
    for i in perm:
        if check_sets(i, singularities):
            continue
        
        next_set = {i}
        while perm[i] not in next_set:
            i = perm[i]
            next_set.add(i)
        singularities.append(next_set)
    return singularities

def find_element_in_partition(partition, element):
    """Gives the singularity corresponding to the right vertex of the saddle."""
    for i, part in enumerate(partition):
        if element in part:
            return i

def collapse_same_genus(cylinder_diagram, cylinder_class):
    """Tells whether a certain collapse will be in the same genus.
    
    Return true if the collapse is forced to be the same genus.
    Return false if the collapse may be a lower genus.
    
    cylinder_list is a list of ints corresponding to the cylinder equivalence class being collapsed
    cylinder_diagram is a surface_dynamics.flat_surfaces.separatrix_diagram.CylinderDiagram object that represents a translation surface
    """
    sing_list = compute_singularities(cylinder_diagram.outgoing_edges_perm())
    digraph = nx.DiGraph()
    digraph.add_nodes_from(range(len(sing_list)))
    
    cylinders = cylinder_diagram.cylinders()
    for cylinder in cylinder_class:
        for c_index in cylinder:
            for bot_saddle in cylinders[c_index][0]:
                for top_saddle in cylinders[c_index][1]:
                    start = find_element_in_partition(sing_list, bot_saddle)
                    end = find_element_in_partition(sing_list, top_saddle)
                    digraph.add_edge(start, end)
    # nx.draw(digraph, with_labels=True, font_weight='bold')
    if nx.is_directed_acyclic_graph(digraph):
        return True
    return False

In [4]:
# C = CylinderDiagrams()
# strata = []
# strata.append(AbelianStratum(2, 2).components()[1])
# strata.append(AbelianStratum(3, 1).components()[0])
# strata.append(AbelianStratum(2, 1, 1).components()[0])
# strata.append(AbelianStratum(1, 1, 1, 1).components()[0])

# for H in strata:
#     cylinders = list(C.get_iterator(H, 4))
#     print(H)
#     for cd in cylinders:
#         hom = find_homologous_cylinders(cd)
#         if hom:
#             print(cd, hom)

In [5]:
# C = CylinderDiagrams()
# strata = []
# strata.append(AbelianStratum(2, 2).components()[1])
# strata.append(AbelianStratum(3, 1).components()[0])
# strata.append(AbelianStratum(2, 1, 1).components()[0])
# strata.append(AbelianStratum(1, 1, 1, 1).components()[0])

# stdout = sys.stdout

# for i in range(4, 5):
#     with open(f"c-{i}-homologous.txt", "w") as file:
#         sys.stdout = file
#         for H in strata:
#             try:
#                 cylinders = list(C.get_iterator(H, i))
#                 print(H)
#                 for c in cylinders:
#                     h = find_homologous_cylinders(c)
#                     if h:
#                         print(c, h[0])
#             except ValueError:
#                 continue
#             print()
# sys.stdout = stdout

In [7]:
def save_output(function):
    def function_with_save(*args):
        stdout = sys.stdout
        function(*args)
        sys.stdout = stdout
    return function_with_save

In [21]:
C = CylinderDiagrams()
# H = AbelianStratum(2, 2).components()[1]
# H = AbelianStratum(2, 1, 1).components()[0]
H = AbelianStratum(1, 1, 1, 1).components()[0]
# H = AbelianStratum(7, 1).components()[0]

# H = AbelianStratum(2, 2).components()[1]
num_cylinders = 6
num_classes = [4]


diagram_list = list(C.get_iterator(H, num_cylinders))
print(len(diagram_list))


4


In [23]:
def main():
    i = 0
    for cd in C.get_iterator(H, num_cylinders):
        possible_classes = valid_equivalence_classes(cd, num_classes, free_cylinders=True)
        if possible_classes:
            print(cd, possible_classes)
            i += 1
            print()
main()

(0,1)-(7) (2)-(1) (3)-(0) (4,7)-(5,6) (5)-(4) (6)-(2,3) [{{0, 5}, {1}, {2, 4}, {3}}, {{0, 5}, {1, 4}, {2}, {3}}, {{0, 5}, {1, 3}, {2}, {4}}, {{0, 5}, {1}, {2, 3}, {4}}, {{0}, {1, 3}, {2, 4}, {5}}, {{0}, {1, 4}, {2, 3}, {5}}]

(0,1)-(7) (2)-(0) (3)-(4) (4,7)-(5,6) (5)-(1) (6)-(2,3) [{{0}, {1}, {2}, {3, 4, 5}}, {{0}, {1, 2, 5}, {3}, {4}}, {{0, 2, 3}, {1}, {4}, {5}}, {{0, 1, 4}, {2}, {3}, {5}}, {{0, 5}, {1}, {2, 4}, {3}}, {{0, 5}, {1, 3}, {2}, {4}}, {{0}, {1, 3}, {2, 4}, {5}}]

(0,3)-(6,7) (1,2)-(4,5) (4)-(3) (5)-(0) (6)-(2) (7)-(1) [{{0}, {1}, {2, 4}, {3, 5}}, {{0}, {1}, {2, 5}, {3, 4}}, {{0, 1}, {2, 5}, {3}, {4}}, {{0, 1}, {2}, {3, 5}, {4}}, {{0, 1}, {2}, {3, 4}, {5}}, {{0, 1}, {2, 4}, {3}, {5}}]

(0,3)-(6,7) (1,2)-(4,5) (4)-(1) (5)-(3) (6)-(2) (7)-(0) [{{0}, {1, 5}, {2}, {3, 4}}, {{0}, {1}, {2, 5}, {3, 4}}, {{0, 1}, {2, 5}, {3}, {4}}, {{0, 2}, {1, 5}, {3}, {4}}, {{0, 2}, {1}, {3, 4}, {5}}, {{0, 1}, {2}, {3, 4}, {5}}]

